# Aufgaben zu Imputer / Transformer

## Stufe 0

Stelle sicher, dass Du verstanden hast, wie man Klassen zusammenbaut und welche Rolle die Parameter spielen.(z.B.  Frochte, Kapitel3.2.5)

## Stufe 1

a) Schreibe eine Klasse MyImputer

Für diese Aufgabe legen wir fest, dass wir NAN-Werte dadurch erkennen, dass sie als 0 abgespeichert sind.
(z.B. alter =0 weist darauf hin, dass Informationen über das Alter nicht vorliegen)

Die Klasse soll unterschiedlich agieren , je nach Parameter.
Parameter können sein : 
	strategy="mean", 
	strategy="feature"  (wird erst in Aufgabe 2d) abgearbeitet.
	es sind auch  weitere Parameter möglich.


Die Klasse soll enthalten
	eine Funktion __init__, die die Parameter abspeichert
	eine Funktion fit, die sich einen Datensatz anschaut und Werte berechnet und abspeichert
	eine Funktion transform, 
		die  eine Datenspalte  neu berechnet.

		Falls die Strategie „mean“ ist, 
		soll die neue Spalte überall  den Mittelwert der alten Daten enthalten, 
			an den Stellen , wo vorher fehlender Werte waren.
		Falls die Strategie „feature“ ist, wird an die fehlende Stelle jeweils die Spaltennummer eingesetzt

		Die Funktion gibt den (eventuell) veränderten Datensatz zurück.

b) ??

c)
Schreibe einen Test .
Erfinde dafür eine Matrix, die aus 5x2 Werten besteht. 
Frage nach, wenn du einen Tipp für die Erzeugung der Matrix brauchst.
Simuliere das Nichtvorhandensein von Werten, indem Du an einigen Stellen 0 einbaust.
Lasse den Transformer einen Datensatz zurückgeben, wo kein fehlender Wert mehr auftaucht.


In [64]:
import numpy as np
import random

In [65]:
# My sample data to test the creation of the MyInputer class

a = [1,2,3,4,5,6,7,8,9,10]
b = [2,4,6,8,10,12,14,16,18,20]
c = [1,0,1,0,1,0,1,0,1,0]
label = [10,0,10,10,0,10,10,0,10,10]

size = [1, 2, 3, 4, 5, 'NaN', 7,8,9,10]
farbe = ['rot','rot','gelb','gelb','blau','blau','rot','rot','rot','rot']
gesund = [1,0,1,0,1,0,1,0,1,0]
label_2 = ['Mini-Apfel','...']

data = np.array([a,b,c,label], dtype=float).T # .T is a shorthand for the transpose of an array. When applied to a 2D array, it swaps the rows and columns.

In [66]:
data

array([[ 1.,  2.,  1., 10.],
       [ 2.,  4.,  0.,  0.],
       [ 3.,  6.,  1., 10.],
       [ 4.,  8.,  0., 10.],
       [ 5., 10.,  1.,  0.],
       [ 6., 12.,  0., 10.],
       [ 7., 14.,  1., 10.],
       [ 8., 16.,  0.,  0.],
       [ 9., 18.,  1., 10.],
       [10., 20.,  0., 10.]])

In [67]:
# Add one NaN in each of the first three columns

random.seed(42)

for col in range(data.shape[1] - 1):  # Exclude the label column
    row = random.randint(0, data.shape[0] - 1)  # Random row index
    data[row, col] = np.nan

In [68]:
data

array([[ 1., nan,  1., 10.],
       [nan,  4.,  0.,  0.],
       [ 3.,  6.,  1., 10.],
       [ 4.,  8.,  0., 10.],
       [ 5., 10., nan,  0.],
       [ 6., 12.,  0., 10.],
       [ 7., 14.,  1., 10.],
       [ 8., 16.,  0.,  0.],
       [ 9., 18.,  1., 10.],
       [10., 20.,  0., 10.]])

In [69]:
class MyInputer:
    def __init__(self, strategy = 'mean'):
        self.strategy = strategy
        self.impute_values = [] # initialize the attribute that will store the value to be substituted

    def fit(self, X):
        # Calculate mean for each column if strategy is mean
        if self.strategy == 'mean':
            for col in range(X.shape[1]):
                # Extract column, ignoring NaN values
                col_values = [value for value in X[:, col] if not np.isnan(value)]
                # Calculate mean
                mean_value = sum(col_values) / len(col_values) if col_values else 0
                self.impute_values.append(mean_value)
        elif self.strategy == 'feature':
            # Use column number as the imput value
            for col in range(X.shape[1]):
                self.impute_values.append(col)

    def transform(self, X):
        X_imputed = X.copy()
        # Replace NaNs with the computed mean values
        for col in range(X.shape[1]):
            X_imputed[np.isnan(X_imputed[:, col]), col] = self.impute_values[col]
        return X_imputed

In [70]:
inputer = MyInputer(strategy='mean')
inputer.fit(data)
data_imputed = inputer.transform(data)
data_imputed

array([[ 1.        , 12.        ,  1.        , 10.        ],
       [ 5.88888889,  4.        ,  0.        ,  0.        ],
       [ 3.        ,  6.        ,  1.        , 10.        ],
       [ 4.        ,  8.        ,  0.        , 10.        ],
       [ 5.        , 10.        ,  0.44444444,  0.        ],
       [ 6.        , 12.        ,  0.        , 10.        ],
       [ 7.        , 14.        ,  1.        , 10.        ],
       [ 8.        , 16.        ,  0.        ,  0.        ],
       [ 9.        , 18.        ,  1.        , 10.        ],
       [10.        , 20.        ,  0.        , 10.        ]])

In [71]:
inputer = MyInputer(strategy='feature')
inputer.fit(data)
data_imputed = inputer.transform(data)
data_imputed

array([[ 1.,  1.,  1., 10.],
       [ 0.,  4.,  0.,  0.],
       [ 3.,  6.,  1., 10.],
       [ 4.,  8.,  0., 10.],
       [ 5., 10.,  2.,  0.],
       [ 6., 12.,  0., 10.],
       [ 7., 14.,  1., 10.],
       [ 8., 16.,  0.,  0.],
       [ 9., 18.,  1., 10.],
       [10., 20.,  0., 10.]])

Stufe 2
a) Die gleiche Aufgabe, aber mit -999 als Kennzeichen für fehlende Werte
b) Die gleiche Aufgabe, aber mit np.nan als Kennzeichen für fehlende Werte np.nan
c) Diesmal darf eine Liste mit verschiedenen Kennzeichen für fehlende Werte eingegeben werden.
   Jeder fehlende Wert soll durch die gleiche Methode ersetzt werden
d) Baue die Klasse MyTransformer aus, so dass sie 2 Strategien abarbeitet.
Falls die Strategie „feature“ ist, wird an die fehlende Stelle jeweils die Spaltennummer eingesetzt 

In [72]:
class MyInputer2:
    def __init__(self, strategy='mean', substitutes=['NaN', 0, -999]):
        self.strategy = strategy
        self.substitutes = substitutes  # List of substitute values
        self.impute_values = []  # Initialize to store impute values for each column

    def insert_substitutes(self, X):
        X_modified = X.copy()
        for col in range(X.shape[1] - 1):  # Exclude the label column
            # Get a list of random row indices for each substitute in the column
            rows = random.sample(range(X.shape[0]), len(self.substitutes))
            for i, substitute in enumerate(self.substitutes):
                # Insert substitute value; handle 'NaN' as np.nan
                if substitute == 'NaN':
                    X_modified[rows[i], col] = np.nan
                else:
                    X_modified[rows[i], col] = substitute
        return X_modified

    def fit(self, X):
        self.impute_values = []
        if self.strategy == 'mean':
            # Calculate mean for each column, excluding values in substitutes
            for col in range(X.shape[1]):
                col_values = [
                    value for value in X[:, col]
                    if (value not in self.substitutes and not (np.isnan(value) and 'NaN' in self.substitutes))
                ]
                mean_value = sum(col_values) / len(col_values) if col_values else 0
                self.impute_values.append(mean_value)
        elif self.strategy == 'feature':
            # Use column number as the impute value
            for col in range(X.shape[1]):
                self.impute_values.append(col)  # Set column index as impute value

    def transform(self, X):
        X_imputed = X.copy()
        # Replace NaNs and other substitutes with the computed impute values
        for col in range(X.shape[1]):
            for substitute in self.substitutes:
                if substitute == 'NaN':
                    # Replace np.nan values
                    X_imputed[np.isnan(X_imputed[:, col]), col] = self.impute_values[col]
                else:
                    # Replace other specified substitute values
                    X_imputed[X_imputed[:, col] == substitute, col] = self.impute_values[col]
        return X_imputed


In [73]:
data = np.array([a, b, c, label], dtype=float).T
data

array([[ 1.,  2.,  1., 10.],
       [ 2.,  4.,  0.,  0.],
       [ 3.,  6.,  1., 10.],
       [ 4.,  8.,  0., 10.],
       [ 5., 10.,  1.,  0.],
       [ 6., 12.,  0., 10.],
       [ 7., 14.,  1., 10.],
       [ 8., 16.,  0.,  0.],
       [ 9., 18.,  1., 10.],
       [10., 20.,  0., 10.]])

In [74]:
inputer = MyInputer2(strategy='mean', substitutes=['NaN', 0, -999])
data_with_substitutes = inputer.insert_substitutes(data)
data_with_substitutes

array([[   1.,    2., -999.,   10.],
       [   2.,   nan,    0.,    0.],
       [-999.,    6.,    1.,   10.],
       [  nan,    8.,    0.,   10.],
       [   5.,   10.,    1.,    0.],
       [   6.,   12.,    0.,   10.],
       [   7.,   14.,    0.,   10.],
       [   8.,   16.,    0.,    0.],
       [   9.,    0.,    1.,   10.],
       [   0., -999.,   nan,   10.]])

In [76]:
inputer.fit(data_with_substitutes)
data_imputed = inputer.transform(data_with_substitutes)
data_imputed

array([[ 1.        ,  2.        ,  1.        , 10.        ],
       [ 2.        ,  9.71428571,  1.        , 10.        ],
       [ 5.42857143,  6.        ,  1.        , 10.        ],
       [ 5.42857143,  8.        ,  1.        , 10.        ],
       [ 5.        , 10.        ,  1.        , 10.        ],
       [ 6.        , 12.        ,  1.        , 10.        ],
       [ 7.        , 14.        ,  1.        , 10.        ],
       [ 8.        , 16.        ,  1.        , 10.        ],
       [ 9.        ,  9.71428571,  1.        , 10.        ],
       [ 5.42857143,  9.71428571,  1.        , 10.        ]])

There is a conceptual error in the exercise definition and the dataset provided by the teacher. Zeros are considered values to be substituted, even though both the label and columns use zeros and binary values as valid and meaningful data. Nevertheless, the code is written according to the specified evaluation criteria.

## Stufe 3

Schreibe eine Klasse Gruppenimputer.
Die Klasse soll kategoriale Zielvariablen haben.
Die Klasse soll den Datensatz in mehrere Untergruppen spalten, bei denen jeweils y denselben Wert hat.
Fehlende Werte sollen durch den Mittelwert der jeweiligen Gruppe ersetzt werden.

a = [1,3,2,'NaN',50,60]
b = [10,12,'NaN',50,55,56]
label = ['Apfel','Apfel','Apfel','Birne','Birne','Birne']

Die Ersatzwerte werden für die Kategorie „Birne“ und „Apfel“ getrennt ermittelt


In [77]:
class MyInputer3:
    def __init__(self, strategy='mean', substitutes=['NaN']):
        self.strategy = strategy
        self.substitutes = substitutes  # List of substitute values
        self.impute_values = {}  # Dictionary to store impute values for each category in the label

    def fit(self, X, labels):
        categories = np.unique(labels)  # Get unique categories from the label
        for category in categories:
            self.impute_values[category] = []
            # Filter data by category
            category_indices = [i for i, label in enumerate(labels) if label == category]
            for col in range(X.shape[1]):
                # Extract column values for the current category, excluding substitutes
                col_values = [
                    X[i, col] for i in category_indices
                    if not np.isnan(X[i, col]) and X[i, col] not in self.substitutes
                ]
                # Calculate mean if strategy is 'mean'
                mean_value = sum(col_values) / len(col_values) if col_values else 0
                self.impute_values[category].append(mean_value)

    def transform(self, X, labels):
        X_imputed = X.copy()
        for i, label in enumerate(labels):
            for col in range(X.shape[1]):
                for substitute in self.substitutes:
                    if substitute == 'NaN' and np.isnan(X_imputed[i, col]):
                        X_imputed[i, col] = self.impute_values[label][col]
                    elif X_imputed[i, col] == substitute:
                        X_imputed[i, col] = self.impute_values[label][col]
        return X_imputed

In [79]:
# Example usage
a = [1, 3, 2, 'NaN', 50, 60]
b = [10, 12, 'NaN', 50, 55, 56]
label = ['Apfel', 'Apfel', 'Apfel', 'Birne', 'Birne', 'Birne']

# Prepare the data
data = np.array([a, b], dtype=float).T
labels = np.array(label)

In [80]:
inputer = MyInputer3(strategy='mean', substitutes=['NaN'])
inputer.fit(data, labels)
data_imputed = inputer.transform(data, labels)

data


array([[ 1., 10.],
       [ 3., 12.],
       [ 2., nan],
       [nan, 50.],
       [50., 55.],
       [60., 56.]])

In [81]:
data_imputed

array([[ 1., 10.],
       [ 3., 12.],
       [ 2., 11.],
       [55., 50.],
       [50., 55.],
       [60., 56.]])